In [3]:
%run "wall_of_imports.ipynb"
%run "agent_state.ipynb"

In [4]:
def information_agent(state: AgentState) -> dict:
    """Answers general travel questions using RAG."""
    try:
        # This agent directly uses the RAG chain to provide travel information
        rag_chain = setup_rag_chain()
        result = rag_chain.invoke({"input": state.query})  # Changed from query to input
        
        # Enhance RAG response with additional context if needed
        enhancement_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a knowledgeable travel information specialist. Review and enhance
            the retrieved information to provide a comprehensive, accurate response to the user's query.
            
            If the retrieved information is incomplete, add relevant details from your knowledge while
            clearly distinguishing between retrieved facts and general knowledge.
            
            Focus on providing practical, useful information that directly addresses the user's needs.
            Include cultural insights, traveler tips, and seasonal considerations when relevant.
            
            Retrieved information:
            {rag_response}
            """),
            ("human", "{input}")  # Changed from query to input
        ])
        
        enhancement_chain = enhancement_prompt | llm | StrOutputParser()
        enhanced_response = enhancement_chain.invoke({
            "input": state.query,
            "rag_response": result.get("answer", "")
        })
        
        return {"agent_response": enhanced_response}
        
    except Exception as e:
        return {"error": str(e)}